In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
import os
from matplotlib.image import imread

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Dropout, LeakyReLU, Flatten, \
    BatchNormalization, Conv2D,Conv2DTranspose, Activation, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
image_path = 'F:\\images\\coffee cup icon'

In [6]:
# widths = []
# heights = []
# for image_filename in os.listdir(image_path + '\\sub'):
    
#     img = imread(image_path + '\\sub' + '\\' + image_filename)
#     width, height, _ = img.shape
#     widths.append(width)
#     heights.append(height)

In [7]:
# sns.jointplot(widths, heights)

In [8]:
# np.median(widths)

In [9]:
# np.median(heights)

In [10]:
image_shapes = (224, 224, 3)
batch_size = 16
dim = 28
depth = 128
input_dim = 100

In [11]:
image_gen = ImageDataGenerator()

In [12]:
image_generator = image_gen.flow_from_directory(
    directory=image_path,
    target_size=image_shapes[:2],
    color_mode='rgb',
    batch_size=batch_size,
    save_format='jpg',
    class_mode=None
)

Found 1028 images belonging to 1 classes.


In [13]:
class DCGAN:
    def __init__(self, image_shapes, batch_size, dim, depth, input_dim, dropout=0.0):
        self.image_shapes = image_shapes
        self.batch_size = batch_size
        self.dim = dim
        self.depth = depth
        self.input_dim = input_dim
        self.dropout = dropout
        
    def generator(self):
        generator = Sequential()

        generator.add(Dense(self.dim * self.dim * self.depth, input_shape=[self.input_dim]))
        generator.add(Reshape((self.dim, self.dim, self.depth)))
        generator.add(BatchNormalization())
        generator.add(Activation('relu'))

        generator.add(Conv2DTranspose(int(self.depth/2), kernel_size=(5, 5), strides=2, padding='same'))
        generator.add(BatchNormalization())
        generator.add(Activation('relu'))

        generator.add(Conv2DTranspose(int(self.depth/4), kernel_size=(5, 5), strides=2, padding='same'))
        generator.add(BatchNormalization())
        generator.add(Activation('relu'))

        generator.add(Conv2DTranspose(3, kernel_size=(5, 5), strides=2, padding='same'))
        generator.add(Activation('tanh'))
        
        return generator
    
    def discriminator(self):
        discriminator = Sequential()
        
        discriminator.add(Conv2D(int(depth/4), kernel_size=(5, 5), strides=2, padding="same",
                                 input_shape=self.image_shapes))
        discriminator.add(LeakyReLU())
        discriminator.add(Dropout(self.dropout))
        
        discriminator.add(Conv2D(depth/2, kernel_size=(5, 5), strides=2, padding="same"))
        discriminator.add(LeakyReLU())
        discriminator.add(Dropout(self.dropout))
        
        discriminator.add(Conv2D(depth, kernel_size=(5, 5), strides=2, padding="same"))
        discriminator.add(LeakyReLU())
        discriminator.add(Dropout(self.dropout))
        
        discriminator.add(Flatten())
        discriminator.add(Dense(1))
        discriminator.add(Activation('sigmoid'))
        
        return discriminator

In [14]:
DCGAN = DCGAN(image_shapes, batch_size, dim, depth, input_dim, dropout=0.2)
generator = DCGAN.generator()
discriminator = DCGAN.discriminator()

In [15]:
# generator.summary()
# discriminator.summary()

In [16]:
optimizer = Adam(learning_rate=0.0002)

In [17]:
# generator.compile(loss='binary_crossentropy',
#                       optimizer=optimizer)

In [18]:
discriminator.compile(loss="binary_crossentropy", optimizer=optimizer)
discriminator.trainable = False

In [19]:
GAN = Sequential([generator, discriminator])

In [20]:
GAN.compile(loss="binary_crossentropy", optimizer=optimizer)

In [21]:
GAN.layers

In [22]:
GAN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 224, 224, 3)       10394947  
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 358977    
Total params: 10,753,924
Trainable params: 10,394,499
Non-trainable params: 359,425
_________________________________________________________________


In [23]:
def train_dcgan(epochs):
    
    generator, discriminator = GAN.layers
    number_of_batches = len(image_generator)

    # Variables that will be used to plot the losses from the discriminator and
    # the adversarial models
    adversarial_loss = np.empty(shape=1)
    discriminator_loss = np.empty(shape=1)
    batches = np.empty(shape=1)

    # Allo plot updates inside for loop
    plt.ion()

    current_batch = 0

    # Let's train the DCGAN for n epochs
    for epoch in range(1, epochs+1):

        print("Epoch " + str(epoch) + "/" + str(epochs) + " :")

        for batch_number in range(number_of_batches):

            start_time = time.time()

            # Get the current batch and normalize the images between -1 and 1
            real_images = image_generator.next()
            real_images /= 127.5
            real_images -= 1

            # The last batch is smaller than the other ones, so we need to
            # take that into account
            current_batch_size = real_images.shape[0]

            # Generate noise
            noise = tf.random.normal(shape=[current_batch_size, input_dim])

            # Generate images
            generated_images = generator.predict(noise)
            
            X_fake_vs_real = tf.concat([generated_images, tf.dtypes.cast(real_images,tf.float32)], axis=0)
            
            y = tf.constant([[0.]] * current_batch_size + [[1.]] * current_batch_size)

            # Let's train the discriminator
            discriminator.trainable = True

            d_loss = discriminator.train_on_batch(X_fake_vs_real, y)

            discriminator_loss = np.append(discriminator_loss, d_loss)

            # Now it's time to train the generator
            discriminator.trainable = False

            noise = tf.random.normal(shape=[current_batch_size * 2, input_dim])

            # We try to mislead the discriminator by giving the opposite labels
            fake_y = np.ones(current_batch_size * 2)

            g_loss = GAN.train_on_batch(noise, fake_y)
            adversarial_loss = np.append(adversarial_loss ,g_loss)
            batches = np.append(batches, current_batch)

            # Each 50 batches show and save images
#             if (batch_number + 1) % 10 == 0:
#                 save_generated_images(generated_images, epoch, batch_number)

            time_elapsed = time.time() - start_time

            # Display and plot the results
            print("     Batch " + str(batch_number + 1) + "/" +
                  str(number_of_batches) +
                  " generator loss | discriminator loss : " +
                  str(g_loss) + " | " + str(d_loss) + ' - batch took ' +
                  str(time_elapsed) + ' s.')

            current_batch += 1

        # Save the model weights each 5 epochs
        if epoch % 20 == 0:
            discriminator.trainable = True
            generator.save('models/generator_epoch_' + str(epoch) + '_' + str(adversarial_loss[-1]) + '.h5')
            discriminator.save('models/discriminator_epoch_' +
                               str(epoch)+ '_' + str(discriminator_loss[-1]) + '.h5')

        # Each epoch update the loss graphs
        plt.figure(1)
        plt.plot(batches, adversarial_loss, color='green',
                 label='Generator Loss')
        plt.plot(batches, discriminator_loss, color='blue',
                 label='Discriminator Loss')
        plt.title("DCGAN Train")
        plt.xlabel("Batch Iteration")
        plt.ylabel("Loss")
        if epoch == 0:
            plt.legend()
        plt.pause(0.0000000001)
        plt.show()

In [24]:
# train_dcgan(epochs=100)